In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-depot-product-search-relevance/product_descriptions.csv.zip
/kaggle/input/home-depot-product-search-relevance/relevance_instructions.docx
/kaggle/input/home-depot-product-search-relevance/train.csv.zip
/kaggle/input/home-depot-product-search-relevance/sample_submission.csv.zip
/kaggle/input/home-depot-product-search-relevance/attributes.csv.zip
/kaggle/input/home-depot-product-search-relevance/test.csv.zip


In [ ]:
%pip install dssm

In [4]:
from dssm.model import DSSM


In [5]:
filenames

['product_descriptions.csv.zip',
 'relevance_instructions.docx',
 'train.csv.zip',
 'sample_submission.csv.zip',
 'attributes.csv.zip',
 'test.csv.zip']

In [22]:
import pandas as pd
import zipfile

zf = zipfile.ZipFile('/kaggle/input/home-depot-product-search-relevance/train.csv.zip') 
df = pd.read_csv(zf.open('train.csv'), encoding='latin-1')

In [23]:
df


,id,product_uid,product_title,search_term,relevance
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67
...,...,...,...,...,...
74062,221457,206638,Atlantic Windowpane 576 CD or 192 DVD Blu-Ray ...,tv riser glass,1.00
74063,221458,206639,Philips 40-Watt Halogen R20 Flood Light Bulb (...,r20 halogen light,3.00
74064,221463,206641,Schlage Camelot In-Active Aged Bronze Handlese...,schlage lock siena half dummy knob with,2.33
74065,221471,206648,Plastec 11 in. x 24 in. Rose Garden Wall Decor...,zen garden decor,3.00


In [31]:
queries = df['search_term'].tolist() # query list, words need to be segmented in advance, and tokens should be spliced with spaces.
documents = df['product_title'].tolist()  # document list, words need to be segmented in advance, and tokens should be spliced with spaces.


In [35]:
model = DSSM('dssm-model', device='cpu', lang='en')


In [36]:
model.fit(queries, documents)


/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/opt/conda/lib/python3.7/site-packages/dssm/dataset.py:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:210.)
  doc_ns = torch.FloatTensor(self.doc_ns[index])
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:372: UserWarnin

In [47]:
from sklearn.metrics.pairwise import cosine_similarity

text_left = 'angle bracket'
text_right = 'Simpson Strong-Tie 12-Gauge Angle'
model = DSSM('dssm-model', device='cpu')
vectors = model.encode([text_left, text_right])
score = cosine_similarity([vectors[0]], [vectors[1]])
print(score)


[[0.30137493]]


/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
